In [68]:
from bsf_env import init_spark, init_mariadb_engine,set_spark_verbosity
from bsf_dbutilities import DBUtils
from pyspark.sql import functions as F
import pandas as pd

try:
    spark.stop()
except NameError:
    # SparkSession doesn't exist
    pass

spark = init_spark("bsf_utilities", log_level="WARN", show_progress=False, enable_ui=True,process_option='manual')
engine = init_mariadb_engine()
ingest_ts = spark.sql("SELECT current_timestamp()").collect()[0][0]

[Spark] Started 'bsf_utilities' log_level=WARN (effective=WARN), progress=False


In [69]:
# ─── Setup Database Communications ──────────────────────────────────────
db = DBUtils(spark, ingest_ts)
db.spark_stats()

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 200)         # Expand width to avoid line breaks


from delta.tables import DeltaTable
import datetime, os
from pyspark.sql import functions as F

# List all databases
databases = [db.name for db in spark.catalog.listDatabases()]

for database in databases:
    print(f"\n=== Database: {database} ===\n")
    
    # Get all tables in this database
    tables = spark.catalog.listTables(database)
    if not tables:
        print("No tables found.")
        continue

    table_info = []
    
    for t in tables:
        full_table_name = f"{database}.{t.name}"
        row_count = spark.table(full_table_name).count()
        table_type = t.tableType
    
        # Default: no max date
        from pyspark.sql import functions as F
        import datetime
        import os
        
        max_date = None
        try:
            df_table = spark.table(full_table_name)
            
            if "LastLoadDate" in df_table.columns:
                max_date = df_table.agg(F.max("LastLoadDate")).collect()[0][0]
                field = "LastLoadDate"
            elif "StockDate" in df_table.columns:
                max_date = df_table.agg(F.max("StockDate")).collect()[0][0]
                field = "StockDate"
            elif "ChangeDate" in df_table.columns:
                max_date = df_table.agg(F.max("ChangeDate")).collect()[0][0]
                field = "ChangeDate"
            else:
                try:
                    dt = DeltaTable.forName(spark, full_table_name)
                    history = dt.history(1)
                    max_date = history.select(F.max("timestamp")).collect()[0][0]
                    field = "timestamp"
                except:
                    max_date = None
        
            # --- Normalize to datetime ---
            if isinstance(max_date, datetime.date) and not isinstance(max_date, datetime.datetime):
                max_date = datetime.datetime.combine(max_date, datetime.time.min)
        
        except:
            # Non-Delta fallback
            table_location = (
                spark.sql(f"DESCRIBE FORMATTED {full_table_name}")
                     .filter("col_name='Location'")
                     .select("data_type")
                     .collect()[0][0]
            )
            max_date = datetime.datetime.fromtimestamp(os.path.getmtime(table_location))


    
        table_info.append((t.name, row_count, field, max_date, table_type))
    
    df_table_info = spark.createDataFrame(
        table_info,
        schema=["TableName", "RowCount","DateField", "MaxDate", "TableType"]
    )
    
    df_table_info.show(truncate=False)
    # 🚨 Safety stop — prevents accidental full execution
    raise RuntimeError("⚠️ This notebook is blocked. Do NOT run all cells without checking!")

    🔍 === Runtime Config Check ===
      📌 User                       : jupyter
      📌 Python Version             : 3.9.21
    🔍 === Spark Runtime Config Check ===
      📌 Name                       : bsf_utilities
      📌 Master                     : spark://10.20.10.19:7077
      📌 Version                    : Not set
      📌 Max Cores                  : 2
      📌 Executor Instances         : 2
      📌 Executor Cores             : 1
      📌 Task Cous                  : 1
      📌 Executor Memory            : 1024m
      📌 Driver Memory              : 2048m
      📌 JVM Memory Overhead        : 256m
      📌 Dynamic Allocation Enabled : true
      📌 Default Parallelism        : 12
      📌 SQL Shuffle Partitions     : 12
      📌 Memory Fraction            : Not set
      📌 Memory StorageFraction     : Not set
      📌 SQL Adaptive               : Not set
      📌 Shuffle Input Size         : Not set
      📌 Delta OptimizeWrite        : Not set
      📌 Delta AutoCompact          : Not set
 

25/09/29 11:47:29 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/09/29 11:47:29 WARN HiveConf: HiveConf of name hive.metastore.client.connect.timeout does not exist
25/09/29 11:47:29 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist



=== Database: bsf ===



25/09/29 11:47:30 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+-----------------------------+--------+----------+--------------------------+---------+
|TableName                    |RowCount|DateField |MaxDate                   |TableType|
+-----------------------------+--------+----------+--------------------------+---------+
|company                      |31051   |ChangeDate|2025-09-25 09:04:36.828567|MANAGED  |
|companyfundamental           |46148   |timestamp |2025-09-29 08:22:33.708   |MANAGED  |
|companystockhistory          |9674    |StockDate |2025-09-24 00:00:00       |MANAGED  |
|companystockhistory_watermark|9674    |StockDate |2025-09-24 00:00:00       |MANAGED  |
|final_candidates             |5       |StockDate |2025-09-24 00:00:00       |MANAGED  |
|final_candidates_enriched    |15      |StockDate |2025-09-24 00:00:00       |MANAGED  |
|history_signals              |38696   |StockDate |2025-09-24 00:00:00       |MANAGED  |
|history_signals_last         |38696   |StockDate |2025-09-24 00:00:00       |MANAGED  |
|signaldriver        

RuntimeError: ⚠️ This notebook is blocked. Do NOT run all cells without checking!

In [73]:
spark.stop()

In [74]:
sdf = spark.table("bsf.company")
sdf.toPandas().describe()


25/09/29 12:15:55 WARN DeltaLog: Failed to parse file:/srv/lakehouse/tables/bsf.db/company/_delta_log/_last_checkpoint. This may happen if there was an error during read operation, or a file appears to be partial. Sleeping and trying again.
java.lang.NullPointerException: Cannot invoke "org.apache.spark.SparkEnv.conf()" because the return value of "org.apache.spark.SparkEnv$.get()" is null
	at org.apache.spark.sql.delta.storage.DelegatingLogStore.<init>(DelegatingLogStore.scala:38)
	at org.apache.spark.sql.delta.storage.LogStore$.createLogStoreWithClassName(LogStore.scala:288)
	at org.apache.spark.sql.delta.storage.LogStoreProvider.createLogStore(LogStore.scala:385)
	at org.apache.spark.sql.delta.storage.LogStoreProvider.createLogStore$(LogStore.scala:380)
	at org.apache.spark.sql.delta.storage.LogStore$.createLogStore(LogStore.scala:266)
	at org.apache.spark.sql.delta.storage.LogStore$.apply(LogStore.scala:279)
	at org.apache.spark.sql.delta.storage.LogStore$.apply(LogStore.scala:274)

Py4JJavaError: An error occurred while calling o2544.table.
: org.sparkproject.guava.util.concurrent.UncheckedExecutionException: java.lang.NullPointerException: Cannot invoke "org.apache.spark.SparkEnv.conf()" because the return value of "org.apache.spark.SparkEnv$.get()" is null
	at org.sparkproject.guava.cache.LocalCache$Segment.get(LocalCache.java:2263)
	at org.sparkproject.guava.cache.LocalCache.get(LocalCache.java:4000)
	at org.sparkproject.guava.cache.LocalCache$LocalManualCache.get(LocalCache.java:4789)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalog.getCachedPlan(SessionCatalog.scala:209)
	at org.apache.spark.sql.execution.datasources.FindDataSourceTable.org$apache$spark$sql$execution$datasources$FindDataSourceTable$$readDataSourceTable(DataSourceStrategy.scala:245)
	at org.apache.spark.sql.execution.datasources.FindDataSourceTable$$anonfun$apply$2.applyOrElse(DataSourceStrategy.scala:285)
	at org.apache.spark.sql.execution.datasources.FindDataSourceTable$$anonfun$app

In [71]:
from pyspark.sql import functions as F

sdf = spark.table("bsf.history_signals")

# Count rows per timeframe
sdf.groupBy("TimeFrame") \
   .agg(F.count("*").alias("row_count")) \
   .orderBy("TimeFrame") \
   .show(truncate=False)


+---------+---------+
|TimeFrame|row_count|
+---------+---------+
|Daily    |9674     |
|Long     |9674     |
|Short    |9674     |
|Swing    |9674     |
+---------+---------+



In [67]:
from delta.tables import DeltaTable
dt = DeltaTable.forName(spark, "bsf.history_signals")
dt.vacuum(retentionHours=0)  # ⚠ deletes unreferenced files immediately


IllegalArgumentException: requirement failed: Are you sure you would like to vacuum files with such a low retention period? If you have
writers that are currently writing to this table, there is a risk that you may corrupt the
state of your Delta table.

If you are certain that there are no operations being performed on this table, such as
insert/upsert/delete/optimize, then you may turn off this check by setting:
spark.databricks.delta.retentionDurationCheck.enabled = false

If you are not sure, please use a value not less than "168 hours".
       

25/09/29 11:41:07 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
25/09/29 11:41:07 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:978)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [66]:
from delta.tables import DeltaTable
dt = DeltaTable.forName(spark, "bsf.history_signals")
dt.history().show(truncate=False)



+-------+-----------------------+------+--------+---------------------------------+-------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------+------------+--------------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                                              |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                |userMetadata|engineInfo                            |
+-------+-----------------------+------+--------+---------------------------------+-------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+--------------------------

In [70]:
spark.read.format("delta").load("/srv/lakehouse/tables/bsf.db/history_signals") \
     .select("TimeFrame").distinct().show()



+---------+
|TimeFrame|
+---------+
|    Short|
|    Swing|
|    Daily|
|     Long|
+---------+



In [33]:
count = spark.sql(f"""
    SELECT COUNT(DISTINCT CompanyId) as cnt
    FROM bsf.company
    WHERE ListingExchange IN (1,2,3,16)
      AND Active = 1
      AND LastClose < 0.1
      AND LastHistoryDate >= date_sub(current_date(), 30)
""").collect()[0]["cnt"]

print(count)




2957


In [31]:
sdf = spark.table("bsf.companystockhistory")
sdf.toPandas().describe()

,CompanyId,Open,High,Low,Close,Volume
count,9674.000000,9674.000000,9674.000000,9674.000000,9674.000000,9.674000e+03
mean,283131.323444,0.202219,0.214481,0.190078,0.200451,7.996915e+05
std,27379.690969,0.629919,0.658202,0.596637,0.621049,8.924417e+06
min,246458.000000,0.000001,0.000001,0.000001,0.000001,0.000000e+00
25%,250003.000000,0.013200,0.014000,0.011900,0.013542,0.000000e+00
50%,289829.000000,0.037925,0.041070,0.035000,0.038220,6.948000e+03
75%,292453.000000,0.090000,0.100000,0.081500,0.090000,1.032100e+05
max,322964.000000,8.500000,9.140100,8.080000,8.680000,6.647880e+08


In [17]:
sdf = spark.table("bsf.companystockhistory")
counts_df = (
    sdf.groupBy("CompanyId")
      .agg(F.count("*").alias("row_count"))
      .orderBy("CompanyId")
)

counts_df.show(50, truncate=False)
#pdf.head(10).T

+---------+---------+
|CompanyId|row_count|
+---------+---------+
|246458   |65       |
|247632   |166      |
|247699   |171      |
|247700   |171      |
|247709   |166      |
|248157   |160      |
|248489   |152      |
|248509   |153      |
|249416   |149      |
|249434   |150      |
|249824   |138      |
|249861   |139      |
|249888   |141      |
|249943   |280      |
|249991   |144      |
|250003   |139      |
|250006   |280      |
|250473   |135      |
|250567   |114      |
|251019   |131      |
|251108   |119      |
|280298   |56       |
|280304   |77       |
|280325   |75       |
|280339   |279      |
|285312   |104      |
|285313   |104      |
|285324   |279      |
|285342   |81       |
|285354   |40       |
|286522   |123      |
|289761   |96       |
|289801   |64       |
|289827   |118      |
|289828   |44       |
|289829   |217      |
|291151   |31       |
|291176   |133      |
|291186   |280      |
|291846   |280      |
|292065   |280      |
|292074   |105      |
|292076   

In [16]:
sdf = spark.table("bsf.companystockhistory_watermark")
pdf = sdf.limit(10).toPandas()
pdf

#pdf.head(10).T

,CompanyId,StockDate,Open,High,Low,Close,Volume
0,249824,2025-07-31,0.000001,0.000001,0.000001,0.000001,0.0
1,249824,2025-08-01,0.000001,0.000001,0.000001,0.000001,0.0
2,249824,2025-08-04,0.000001,0.000001,0.000001,0.000001,0.0
3,249824,2025-08-05,0.000001,0.000001,0.000001,0.000001,0.0
4,249824,2025-08-06,0.000001,0.000001,0.000001,0.000001,0.0
5,249824,2025-08-07,0.000001,0.000001,0.000001,0.000001,0.0
6,249824,2025-08-08,0.000001,0.000001,0.000001,0.000001,0.0
7,249824,2025-08-11,0.000001,0.000001,0.000001,0.000001,0.0
8,249824,2025-08-12,0.000001,0.000001,0.000001,0.000001,0.0
9,249824,2025-08-13,0.000001,0.000001,0.000001,0.000001,0.0


In [5]:
sdf = spark.table("bsf.history_signals_last")
pdf = sdf.limit(10).toPandas()
pdf

,CompanyId,StockDate,Open,High,Low,Close,TomorrowClose,Return,TomorrowReturn,MA,MA_slope,UpTrend_MA,DownTrend_MA,MomentumUp,MomentumDown,ConfirmedUpTrend,ConfirmedDownTrend,Volatility,LowVolatility,HighVolatility,SignalStrength,SignalStrengthHybrid,ActionConfidence,BullishStrengthHybrid,BearishStrengthHybrid,SignalDuration,PatternAction,CandleAction,UpTrend_Return,CandidateAction,Action,TomorrowAction,TimeFrame
0,52,2025-09-22,0.0510,0.05500,0.0350,0.03500,NaN,-0.338999,NaN,0.053590,0.092603,True,False,False,True,False,False,1.530954e-01,False,True,4,1.465623,1.465623,0.798956,1.465623,152.0,Buy,Sell,False,Sell,Sell,Hold,Swing
1,145,2025-09-22,0.0190,0.01900,0.0190,0.01900,NaN,0.000000,NaN,0.019000,-0.045226,False,True,False,True,False,True,1.430176e-02,False,False,0,0.892754,0.000000,0.892754,0.892754,86.0,Sell,Hold,False,Buy,Hold,Hold,Swing
2,641,2025-09-22,0.0135,0.01350,0.0135,0.01350,NaN,0.000000,NaN,0.012000,0.287554,True,False,True,False,True,False,2.539276e-01,False,True,0,0.709860,0.000000,0.709860,0.709860,193.0,Buy,Hold,True,Buy,Hold,Hold,Swing
3,972,2025-09-22,0.0290,0.02900,0.0290,0.02900,NaN,0.000000,NaN,0.029000,0.000000,False,False,False,False,False,False,1.000000e-08,False,False,0,0.000000,0.000000,0.000000,0.000000,61.0,Hold,Hold,False,Hold,Hold,Hold,Swing
4,1394,2025-09-22,0.0031,0.00335,0.0031,0.00335,NaN,-0.042857,NaN,0.003176,-0.009357,False,True,True,False,False,False,6.496555e-02,True,False,1,0.399176,0.399176,0.399176,0.399176,175.0,Sell,Hold,True,Sell,Sell,Hold,Swing
5,1493,2025-09-22,0.0013,0.00130,0.0013,0.00130,NaN,0.000000,NaN,0.001300,0.048387,True,False,False,False,False,False,2.635231e-02,True,False,2,0.634120,0.000000,0.634120,0.634120,127.0,Buy,Hold,False,Buy,Hold,Hold,Swing
6,1849,2025-09-22,0.0410,0.04300,0.0410,0.04300,NaN,0.000000,NaN,0.042418,0.020890,True,False,True,False,True,False,5.315796e-02,True,False,0,0.481975,0.000000,0.481975,0.481975,174.0,Buy,Hold,True,Buy,Hold,Hold,Swing
7,1851,2025-09-22,0.0099,0.00990,0.0099,0.00990,NaN,0.000000,NaN,0.007380,-0.102190,False,True,True,False,False,False,2.814933e-01,False,True,2,0.678016,0.344683,0.678016,0.344683,130.0,Sell,Buy,True,Sell,Sell,Hold,Swing
8,2217,2025-09-22,0.1000,0.10950,0.0900,0.09660,NaN,-0.086093,NaN,0.109460,0.081065,True,False,False,True,False,False,1.769237e-01,True,False,1,0.378258,0.378258,0.378258,0.378258,159.0,Buy,Hold,False,Buy,Buy,Hold,Swing
9,2285,2025-09-22,0.0214,0.02700,0.0202,0.02700,NaN,0.273585,NaN,0.023380,0.035430,True,False,True,False,True,False,2.480436e-01,True,False,1,0.414802,0.414802,0.414802,0.414802,163.0,Buy,Hold,True,Buy,Buy,Hold,Swing


In [37]:
sdf = spark.table("bsf.signaldriver")
sdf.toPandas().describe()



,CompanyId,Open,High,Low,Close,Volume,PeRatio,PegRatio,PbRatio,ReturnOnEquity,GrossMarginTTM,NetProfitMarginTTM,TotalDebtToEquity,CurrentRatio,InterestCoverage,EpsChangeYear,RevChangeYear,Beta,ShortIntToFloat
count,9674.000000,9674.000000,9674.000000,9674.000000,9674.000000,9.674000e+03,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.0,3276.0,3276.000000,3276.000000,3276.0
mean,283131.323444,0.202219,0.214481,0.190078,0.200451,7.996915e+05,-1.144439,1.239174,-9.447311,346.364976,-674.503793,-7628.716447,-39.978170,1.334098,0.0,0.0,-0.468582,1.020204,0.0
std,27379.690969,0.629919,0.658202,0.596637,0.621049,8.924417e+06,4.105461,20.249965,49.670092,1143.552225,2987.555880,14981.101512,582.658094,2.312820,0.0,0.0,85.455414,5.562771,0.0
min,246458.000000,0.000001,0.000001,0.000001,0.000001,0.000000e+00,-32.894700,-7.452120,-214.492000,-1684.130000,-15489.800000,-66019.900000,-9110.730000,0.000000,0.0,0.0,-77.956700,-5.535060,0.0
25%,250003.000000,0.013200,0.014000,0.011900,0.013542,0.000000e+00,-0.046680,-0.001370,0.000000,-73.596500,-87.017100,-7555.470000,0.000000,0.025240,0.0,0.0,0.000000,0.000000,0.0
50%,289829.000000,0.037925,0.041070,0.035000,0.038220,6.948000e+03,0.000000,0.000000,0.000000,-10.043000,0.000000,-22.427900,0.000000,0.740420,0.0,0.0,0.000000,0.505670,0.0
75%,292453.000000,0.090000,0.100000,0.081500,0.090000,1.032100e+05,0.000000,0.003530,0.601000,0.000000,12.892500,0.000000,13.492800,1.166000,0.0,0.0,0.000000,1.441880,0.0
max,322964.000000,8.500000,9.140100,8.080000,8.680000,6.647880e+08,1.486670,321.708000,250.000000,3739.110000,100.000000,356.913000,457.778000,13.995200,0.0,0.0,1295.330000,85.782400,0.0


In [50]:
sdf = spark.table("bsf.history_signals")
pdf = sdf.limit(10).toPandas()
pdf

,UserId,CompanyId,StockDate,Open,High,Low,Close,TomorrowClose,Return,TomorrowReturn,MA,MA_slope,UpTrend_MA,DownTrend_MA,MomentumUp,MomentumDown,ConfirmedUpTrend,ConfirmedDownTrend,Volatility,LowVolatility,HighVolatility,SignalStrength,SignalStrengthHybrid,ActionConfidence,BullishStrengthHybrid,BearishStrengthHybrid,SignalDuration,PatternAction,CandleAction,UpTrend_Return,CandidateAction,Action,TomorrowAction,TimeFrame
0,1,249991,2025-09-02,0.0800,0.1094,0.070000,0.099600,0.072635,0.383333,-0.270733,0.078122,0.075762,True,False,True,False,True,False,0.139085,True,False,0.0,1.000000,0.795773,1.000000,1.000000,18.0,Hold,Buy,True,Buy,Hold,Sell,Long
1,1,249991,2025-09-03,0.0900,0.1000,0.072635,0.072635,0.100000,-0.270733,0.376747,0.077385,0.050717,True,False,False,True,False,True,0.162748,False,True,2.4,0.975007,0.780777,0.475007,0.975007,18.0,Hold,Hold,False,Sell,Sell,Buy,Long
2,1,249991,2025-09-04,0.0700,0.1000,0.050000,0.100000,0.089900,0.376747,-0.101000,0.079715,0.072596,True,False,True,False,True,False,0.199934,False,True,1.2,0.829140,0.693257,0.829140,0.829140,18.0,Hold,Hold,True,Buy,Buy,Sell,Long
3,1,249991,2025-09-05,0.0990,0.0990,0.089900,0.089900,0.070000,-0.101000,-0.221357,0.080604,0.083069,True,False,True,False,True,False,0.204937,False,True,1.2,0.644575,0.582518,0.644575,0.644575,19.0,Hold,Hold,True,Sell,Sell,Hold,Long
4,1,249991,2025-09-08,0.0808,0.0950,0.070000,0.070000,0.077900,-0.221357,0.112857,0.080064,0.053309,True,False,False,True,False,True,0.218150,False,True,1.2,0.368132,0.314192,0.368132,0.368132,19.0,Hold,Hold,False,Sell,Hold,Buy,Long
5,1,249991,2025-09-09,0.0900,0.0948,0.052700,0.077900,0.088000,0.112857,0.129653,0.079974,0.044567,True,False,False,False,False,False,0.220316,False,True,3.0,0.753102,0.545175,0.253102,0.753102,19.0,Hold,Hold,False,Buy,Buy,Hold,Long
6,1,249991,2025-09-10,0.0780,0.0900,0.075010,0.088000,0.088000,0.129653,0.000000,0.081524,0.054344,True,False,True,False,True,False,0.219794,False,True,0.0,0.605236,0.456455,0.605236,0.605236,19.0,Hold,Buy,True,Buy,Hold,Hold,Long
7,1,249991,2025-09-11,0.0650,0.0880,0.065000,0.088000,0.085000,0.000000,-0.034091,0.083064,0.062308,True,False,True,False,True,False,0.219822,False,True,0.0,0.565408,0.432558,0.565408,0.565408,18.0,Hold,Hold,True,Hold,Hold,Sell,Long
8,1,249991,2025-09-12,0.0850,0.0850,0.085000,0.085000,0.065000,-0.034091,-0.235294,0.084304,0.067786,True,False,True,False,True,False,0.220776,False,True,2.4,0.589957,0.447287,0.589957,0.589957,17.0,Hold,Hold,True,Sell,Sell,Hold,Long
9,1,249991,2025-09-15,0.0605,0.0650,0.060500,0.065000,0.079600,-0.235294,0.224615,0.083603,0.083625,True,False,False,True,False,True,0.236992,False,True,1.2,0.864551,0.612064,0.364551,0.864551,17.0,Hold,Hold,False,Sell,Hold,Buy,Long


In [8]:
sdf = spark.table("bsf.final_candidates")
pdf = sdf.limit(10).toPandas()
pdf

,CompanyId,TimeFrame,StockDate,Open,High,Low,Close,TomorrowClose,Return,TomorrowReturn,MA,MA_slope,UpTrend_MA,DownTrend_MA,MomentumUp,MomentumDown,ConfirmedUpTrend,ConfirmedDownTrend,Volatility,LowVolatility,HighVolatility,SignalStrength,SignalStrengthHybrid,ActionConfidence,BullishStrengthHybrid,BearishStrengthHybrid,SignalDuration,PatternAction,CandleAction,UpTrend_Return,CandidateAction,Action,TomorrowAction,BuyRank,BestModel,BestModel_RMSE,BestModel_MAE,BestModel_MAPE,BestModel_DirectionAcc,BestModel_R2,BestModel_AdjR2,Pred_Sklearn,PredictedReturn_Sklearn,Pred_Linear,PredictedReturn_Linear,Linear_RMSE,Linear_MAE,Linear_MAPE,Linear_DirectionAcc,Linear_R2,Linear_AdjR2,Pred_Lasso,PredictedReturn_Lasso,Lasso_RMSE,Lasso_MAE,Lasso_MAPE,Lasso_DirectionAcc,Lasso_R2,Lasso_AdjR2,Pred_Ridge,PredictedReturn_Ridge,Ridge_RMSE,Ridge_MAE,Ridge_MAPE,Ridge_DirectionAcc,Ridge_R2,Ridge_AdjR2,Pred_XGBoost,PredictedReturn_XGBoost,XGBoost_RMSE,XGBoost_MAE,XGBoost_MAPE,XGBoost_DirectionAcc,XGBoost_R2,XGBoost_AdjR2,MaxPredictedReturn,Phase2_Rank,SARIMAX_PredictedClose,SARIMAX_PredictedReturn,WeightedScore,AIC,MlType,TradingSymbol,Name,Phase3_Rank
0,100167,Daily,2025-09-22,0.0200,0.03000,0.0200,0.02000,NaN,65.666667,NaN,0.010043,0.165837,True,False,True,False,True,False,20.801939,False,True,4,1.999948,0.999948,0.999948,1.999948,41.0,Buy,Hold,True,Buy,Buy,Hold,14,XGBoost,0.000222,0.000117,23.965739,0.924051,0.998176,0.997998,0.059578,1.978905,0.600873,29.043656,0.002445,0.001216,246.314138,0.620253,0.779743,0.758329,0.003931,-0.803441,0.004842,0.004620,1267.855456,0.645570,0.136029,0.052032,0.010645,-0.467759,0.004055,0.003307,923.894960,0.611814,0.393814,0.334879,0.015556,-0.222219,0.000222,0.000117,23.965739,0.924051,0.998176,0.997998,29.043656,1,0.016226,-0.188692,17.350717,-2072.654464,automl,WRCDF,"WIRECARD AG, BERLIN",1
1,92779,Daily,2025-09-22,0.0115,0.01265,0.0115,0.01265,NaN,0.193396,NaN,0.012204,-0.021196,False,True,False,True,False,False,0.207573,True,False,2,1.002751,1.002751,1.002751,0.502751,166.0,Sell,Buy,True,Buy,Buy,Hold,28,XGBoost,0.000830,0.000615,0.049286,0.753571,0.997741,0.997548,0.014707,0.162609,0.013489,0.066329,0.002362,0.001615,0.136591,0.532143,0.981719,0.980160,0.032810,1.593639,0.016537,0.013329,1.174648,0.392857,0.103985,0.027581,0.037355,1.952980,0.011549,0.008866,0.751232,0.428571,0.562974,0.525708,0.012598,-0.004140,0.000830,0.000615,0.049286,0.753571,0.997741,0.997548,1.952980,3,0.012650,0.000000,1.171788,-2287.052933,automl,STGZ,STARGAZE ENTMT GROUP,2
2,59260,Daily,2025-09-22,0.0015,0.00165,0.0015,0.00160,NaN,0.032258,NaN,0.001693,-0.024691,False,True,False,True,False,False,0.050199,True,False,2,1.067693,1.067693,1.067693,0.567693,182.0,Sell,Hold,True,Buy,Buy,Hold,22,Linear,0.000130,0.000083,0.023138,0.818505,0.992884,0.992280,0.001955,0.221996,0.001665,0.040569,0.000130,0.000083,0.023138,0.818505,0.992884,0.992280,0.003036,0.897631,0.001319,0.001031,0.264696,0.362989,0.263236,0.200653,0.002549,0.592897,0.001153,0.000893,0.218106,0.626335,0.437481,0.389699,0.002107,0.316706,0.000168,0.000109,0.030664,0.800712,0.988075,0.987062,0.897631,2,0.001600,0.000000,0.538579,-3603.627631,automl,IGPK,INTEGRATED CANNABIS,3
3,67984,Daily,2025-09-22,0.0539,0.05670,0.0539,0.05670,NaN,0.051948,NaN,0.058197,-0.009964,False,True,False,True,False,False,0.080576,False,True,4,1.491882,1.491882,1.491882,0.491882,165.0,Sell,Buy,True,Buy,Buy,Hold,6,XGBoost,0.002772,0.001885,0.024518,0.857143,0.997127,0.996861,0.057045,0.006088,0.059701,0.052933,0.005245,0.002749,0.032772,0.764479,0.989717,0.988762,0.083729,0.476702,0.051672,0.035789,0.629236,0.478764,0.001945,-0.090701,0.064654,0.140284,0.014377,0.009303,0.125533,0.621622,0.922731,0.915559,0.052742,-0.069798,0.002772,0.001885,0.024518,0.857143,0.997127,0.996861,0.476702,7,0.056432,-0.004718,0.284134,-1555.210275,automl,MATEF,BLOCKMATE VENTURES I,4
4,98577,Daily,2025-09-22,0.0027,0.00270,0.0027,0.00270,NaN,0.038462,NaN,0.002614,0.005495,True,False,True,False,True,False,0.012163,True,False,

In [58]:
sdf = spark.table("bsf.company")
# Group by CompanyId, TimeFrame, StockDate
counts_df = (
    sdf.groupBy("ListingExchange")
      .agg(F.count("*").alias("row_count"))
      .orderBy("ListingExchange")
)

counts_df.show(50, truncate=False)



sdf = spark.table("bsf.history_signals")

# Group by CompanyId, TimeFrame, StockDate
counts_df = (
    sdf.groupBy( "TimeFrame")
      .agg(F.count("*").alias("row_count"))
      .orderBy( "TimeFrame")
)

counts_df.show(50, truncate=False)


25/09/29 11:26:57 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/09/29 11:26:57 WARN HiveConf: HiveConf of name hive.metastore.client.connect.timeout does not exist
25/09/29 11:26:57 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/09/29 11:26:57 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+---------------+---------+
|ListingExchange|row_count|
+---------------+---------+
|1              |6442     |
|2              |3405     |
|16             |21204    |
+---------------+---------+

+---------+---------+
|TimeFrame|row_count|
+---------+---------+
|Long     |9674     |
+---------+---------+



In [11]:
# Assume you have a Spark DataFrame
sdf = spark.table("bsf.companystockhistory")

# 1️⃣ Show the schema (data types included)
sdf.printSchema()

# 2️⃣ Get a list of column names
# print(sdf.columns)

# 3️⃣ Show first few rows with all columns (default shows truncated view)
sdf.show(truncate=False)  

# 4️⃣ For more detailed metadata about columns
sdf.dtypes  # Returns a list of (column_name, data_type)


root
 |-- CompanyId: long (nullable = true)
 |-- StockDate: date (nullable = true)
 |-- OpenPrice: double (nullable = true)
 |-- HighPrice: double (nullable = true)
 |-- LowPrice: double (nullable = true)
 |-- ClosePrice: double (nullable = true)
 |-- StockVolume: double (nullable = true)

['CompanyId', 'StockDate', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice', 'StockVolume']
+---------+----------+---------+---------+--------+----------+-----------+
|CompanyId|StockDate |OpenPrice|HighPrice|LowPrice|ClosePrice|StockVolume|
+---------+----------+---------+---------+--------+----------+-----------+
|64490    |2024-08-28|0.054    |0.054    |0.0535  |0.0535    |3200.0     |
|64490    |2024-08-29|0.0533   |0.0533   |0.0533  |0.0533    |2499.0     |
|64490    |2024-08-30|0.053    |0.053    |0.053   |0.053     |1334.0     |
|64490    |2024-09-03|0.041    |0.053    |0.041   |0.0416    |5275.0     |
|64490    |2024-09-04|0.0416   |0.0416   |0.0416  |0.0416    |0.0        |
|64490    |2024

[('CompanyId', 'bigint'),
 ('StockDate', 'date'),
 ('OpenPrice', 'double'),
 ('HighPrice', 'double'),
 ('LowPrice', 'double'),
 ('ClosePrice', 'double'),
 ('StockVolume', 'double')]

In [10]:
# 🚨 Safety stop — prevents accidental full execution
raise RuntimeError("⚠️ This notebook is blocked. Do NOT run all cells without checking!")


RuntimeError: ⚠️ This notebook is blocked. Do NOT run all cells without checking!